# IMPORT THƯ VIỆN VÀ LLMS

In [1]:
from openai import OpenAI
import tiktoken
import time, json
import google.generativeai as genai
from tqdm.notebook import tqdm

In [2]:
api_key_trung = "secret_key"
client = OpenAI(
    api_key=api_key_trung,
)

GOOGLE_API_KEY = 'secret_key'
genai.configure(api_key=GOOGLE_API_KEY)
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash
models/gemini-1.5-flash-001-tuning


# LOADING DATA

In [3]:
with open('./data/Amr_v4/train_v5.txt', 'rt') as f:
    train = f.read().split('\n\n')
    train_sentences = [_ for _ in train if _.startswith('#::')]
    train_viamr = [_ for _ in train if not _.startswith("#::")]

with open('./data/Amr_v4/test_v5.txt', 'rt') as f:
    test = f.read().split('\n')

len(train_sentences), len(train_viamr), len(test)

# for i, j in zip(train_sentences, train_viamr):
#     print(i, j, sep = '\n')

(252, 252, 369)

In [4]:
prompt_v1 = lambda input_sentence, train_ex: f"""
Hãy chuyển đổi câu sau sang định dạng AMR:

```
{input_sentence}
```

Dưới đây là các ví dụ cụ thể trong đó:
* dòng có ký hiệu ::snt hoặc #snt đầu tiên là chỉ câu input đầu vào của người dùng (tập trung vào phần sau của :: snt)
* đoạn còn lại thì là kết quả sau khi chuyển một câu thành định dạng AMR (để ý kỹ định dạng và cách đưa input sang dạng AMR)

Ví dụ:
```
{train_ex}
```
Lưu ý:

* Hãy đảm bảo rằng định dạng AMR được giữ nguyên.
* Hãy sử dụng các ký hiệu chính xác trong AMR.
* Hãy đảm bảo rằng các từ trong AMR được ánh xạ chính xác với các từ trong câu đầu vào.
"""

train_all = '\n\n'.join(train)

# GPT

In [31]:
def gpt_4o(sent: str):

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt_v1(sent, train_all)
            },
        ],
        model="gpt-4o-mini",
        temperature=0,
    )
    
    result = chat_completion.choices[0].message.content
    status = True if result else False
    if not status:
        raise RuntimeError
    return result

In [32]:
gpt_output = []
for t in test:
    _ = gpt_4o(t)
    gpt_output.append(_)

In [33]:
with open('./llms_output/amr_v4/v5_gpt_4o_few.txt', 'wt') as f:
    for out in gpt_output:
        f.write(out + '\n\n\n')

# GEMINI

In [6]:
model = genai.GenerativeModel('gemini-1.5-flash')
GEMINI_FLAG = '<Error>'

def gemini_viamr(sent: str):
    
    promt = prompt_v1(sent, train_ex = train_all)
    res = model.generate_content(promt)

    return res.text

In [7]:
gemini_output = []
for t in test:
    try:
        _ = gemini_viamr(t)
    except:
        _ = GEMINI_FLAG

    gemini_output.append(_) 
    time.sleep(20)

In [18]:
len(gemini_output), gemini_output.count(GEMINI_FLAG)

(369, 26)

In [17]:
for i, (t, out) in tqdm(enumerate(zip(test, gemini_output)), total= gemini_output.count(GEMINI_FLAG)):
    if out == GEMINI_FLAG:
        try:
            _ = gemini_viamr(t)
        except:
            _ = GEMINI_FLAG
        
        gemini_output[i] = _
        time.sleep(10)

  0%|          | 0/29 [00:00<?, ?it/s]

In [19]:
with open('./llms_output/amr_v4/v5_gemini_outputv1.txt', 'wt') as f:
    for i in gemini_output:
        f.write(i + '\n\n\n')